In [15]:
import cv2
import numpy as np

video_path = "LaneVideo.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ 無法開啟影片")
    exit()

ret, frame = cap.read()
if not ret or frame is None:
    print("❌ 無法讀取影片的第一幀")
    cap.release()
    exit()

orig_height, orig_width = frame.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS)
scale = 0.5

# 設定輸出影片大小
out_width = int(orig_width * scale)
out_height = int(orig_height * scale)

# 建立 VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4 格式
out = cv2.VideoWriter('LaneResult.mp4', fourcc, fps, (out_width, out_height))

# 平滑參數
smooth_factor = 0.2  # 越小越穩定但反應越慢
prev_left_fit, prev_right_fit = None, None

def average_slope_intercept(lines):
    slopes, intercepts = [], []
    for x1, y1, x2, y2 in lines:
        if x2 == x1:
            continue
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        slopes.append(slope)
        intercepts.append(intercept)
    if slopes:
        return np.mean(slopes), np.mean(intercepts)
    return None

def make_line_points(y1, y2, slope, intercept):
    if slope is None:
        return None
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return (x1, int(y1)), (x2, int(y2))

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    try:
        img = cv2.resize(frame, (out_width, out_height))
    except cv2.error:
        continue

    height, width = img.shape[:2]

    # 灰階、模糊、邊緣
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(gray, kernel, iterations=1)
    blur = cv2.GaussianBlur(dilated, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 120)

    # ROI 遮罩
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (int(0.25 * width), height),
        (int(0.35 * width), int(0.8 * height)),
        (int(0.6 * width), int(0.8 * height)),
        (int(0.7 * width), height)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Hough Lines
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, threshold=30, minLineLength=40, maxLineGap=120)

    lane_img = img.copy()
    left_lines, right_lines = [], []

    if lines is not None:
        for x1, y1, x2, y2 in lines[:, 0]:
            if x2 == x1:
                continue
            slope = (y2 - y1) / (x2 - x1)
            if slope < -0.3:
                left_lines.append((x1, y1, x2, y2))
            elif slope > 0.3:
                right_lines.append((x1, y1, x2, y2))

    y_bottom = height
    y_top = int(0.8 * height)
    left_fit = average_slope_intercept(left_lines)
    right_fit = average_slope_intercept(right_lines)

    # 平滑化
    if left_fit is not None:
        if prev_left_fit is not None:
            left_fit = (
                smooth_factor * left_fit[0] + (1 - smooth_factor) * prev_left_fit[0],
                smooth_factor * left_fit[1] + (1 - smooth_factor) * prev_left_fit[1],
            )
        prev_left_fit = left_fit
    else:
        left_fit = prev_left_fit

    if right_fit is not None:
        if prev_right_fit is not None:
            right_fit = (
                smooth_factor * right_fit[0] + (1 - smooth_factor) * prev_right_fit[0],
                smooth_factor * right_fit[1] + (1 - smooth_factor) * prev_right_fit[1],
            )
        prev_right_fit = right_fit
    else:
        right_fit = prev_right_fit

    left_line = make_line_points(y_bottom, y_top, *left_fit) if left_fit else None
    right_line = make_line_points(y_bottom, y_top, *right_fit) if right_fit else None

    # 畫面合成
    overlay = lane_img.copy()
    if left_line and right_line:
        pts = np.array([[left_line[0], left_line[1], right_line[1], right_line[0]]], np.int32)
        cv2.fillPoly(overlay, pts, (0, 255, 0))
        cv2.addWeighted(overlay, 0.4, lane_img, 0.6, 0, lane_img)
        mid_x = (left_line[0][0] + right_line[0][0]) // 2
        mid_y = left_line[0][1]
        cv2.circle(lane_img, (mid_x, mid_y), 6, (0, 0, 255), -1)

    # 顯示畫面
    cv2.imshow("Lane Detection", lane_img)

    # 寫入影片
    out.write(lane_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ 影片已存成 LaneResult.mp4")


✅ 影片已存成 LaneResult.mp4
